In [2]:
import sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import os
import numpy
import ast

In [3]:
hostname='localhost'
dbname = 'congressional_bills'
username = 'melissaferrari'

### Database connection with sqlalchemy.

In [60]:
data_agg.head()

,actions,amendments,bill_id,bill_type,by_request,committee_reports,committees,congress,cosponsors,enacted_as,...,short_title,sponsor,status,status_at,subjects,subjects_top_term,summary,titles,updated_at,url
0,"[{'acted_at': '2015-03-03', 'references': [], ...",[],hr1216-114,hr,False,[],"[{'committee_id': 'HSBA', 'activity': ['referr...",114,"[{'withdrawn_at': None, 'district': '7', 'spon...",NaN,...,Maker-Taker Conflict of Interest Reform Act of...,"{'district': '8', 'type': 'person', 'title': '...",REFERRED,2015-03-03,"['Congressional oversight', 'Finance and finan...",Finance and financial sector,"{'date': '2015-03-03T05:00:00Z', 'text': 'Make...",[{'title': 'Maker-Taker Conflict of Interest R...,2017-12-11T19:28:52Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...
1,"[{'acted_at': '2016-03-16', 'references': [], ...",[],hr4764-114,hr,False,[],"[{'committee_id': 'HSVR', 'activity': ['referr...",114,"[{'withdrawn_at': None, 'district': '12', 'spo...",NaN,...,Puppies Assisting Wounded Servicemembers (PAWS...,"{'district': '6', 'type': 'person', 'title': '...",REFERRED,2016-03-16,"['Armed forces and national security', 'Health...",Armed forces and national security,"{'date': '2016-03-16T04:00:00Z', 'text': 'Pupp...",[{'title': 'Puppies Assisting Wounded Servicem...,2017-12-11T21:02:18Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...
2,"[{'acted_at': '2015-01-13', 'references': [], ...",[],hr337-114,hr,False,[],"[{'committee_id': 'HSPW', 'activity': ['referr...",114,[],NaN,...,Maritime Lien Reform Act,"{'district': None, 'type': 'person', 'title': ...",REFERRED,2015-01-13,"['Civil actions and liability', 'Licensing and...",Transportation and public works,"{'date': '2015-01-13T05:00:00Z', 'text': ""Mari...","[{'title': 'Maritime Lien Reform Act', 'type':...",2017-12-11T19:10:21Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...
3,"[{'acted_at': '2016-09-14', 'references': [], ...",[],hr6033-114,hr,False,[],"[{'committee_id': 'HSIF', 'activity': ['referr...",114,"[{'withdrawn_at': None, 'district': '13', 'spo...",NaN,...,Adding Middle East Respiratory Syndrome to the...,"{'district': '52', 'type': 'person', 'title': ...",REFERRED,2016-09-14,['Health'],Health,"{'date': '2016-09-14T04:00:00Z', 'text': 'Addi...",[{'title': 'Adding Middle East Respiratory Syn...,2017-12-11T21:27:52Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...
4,"[{'acted_at': '2015-05-21', 'references': [], ...",[],hr2493-114,hr,False,[],"[{'committee_id': 'HSAS', 'activity': ['referr...",114,"[{'withdrawn_at': None, 'district': '12', 'spo...",NaN,...,Wounded Warrior Service Dog Act of 2015,"{'district': '2', 'type': 'person', 'title': '...",REFERRED,2015-05-21,['Animal protection and human-animal relations...,Armed forces and national security,"{'date': '2015-05-21T04:00:00Z', 'text': 'Woun...",[{'title': 'Wounded Warrior Service Dog Act of...,2017-12-11T20:19:56Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...


In [4]:
#engine = create_engine('postgres://{}@{}/{}'.format(hostname, username, dbname))
engine = create_engine('postgres://{}/{}'.format(hostname, dbname))

In [5]:
print(engine.url)

postgres://localhost/congressional_bills


In [6]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


### Load in csv data.

In [7]:
def import_csv(file_path):
    df = pd.read_csv(file_path)
    if 'Unnamed: 0' in df.columns:
        del df['Unnamed: 0']
    return df

In [8]:
data_path = '/Users/melissaferrari/Projects/repo/insight/data_collection'
file_name = 'agg_propublica_114hr.csv'
data_agg = import_csv(os.path.join(data_path, file_name))

In [9]:
data_agg.head()

,actions,amendments,bill_id,bill_type,by_request,committee_reports,committees,congress,cosponsors,enacted_as,...,short_title,sponsor,status,status_at,subjects,subjects_top_term,summary,titles,updated_at,url
0,"[{'acted_at': '2015-03-03', 'references': [], ...",[],hr1216-114,hr,False,[],"[{'committee_id': 'HSBA', 'activity': ['referr...",114,"[{'withdrawn_at': None, 'district': '7', 'spon...",NaN,...,Maker-Taker Conflict of Interest Reform Act of...,"{'district': '8', 'type': 'person', 'title': '...",REFERRED,2015-03-03,"['Congressional oversight', 'Finance and finan...",Finance and financial sector,"{'date': '2015-03-03T05:00:00Z', 'text': 'Make...",[{'title': 'Maker-Taker Conflict of Interest R...,2017-12-11T19:28:52Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...
1,"[{'acted_at': '2016-03-16', 'references': [], ...",[],hr4764-114,hr,False,[],"[{'committee_id': 'HSVR', 'activity': ['referr...",114,"[{'withdrawn_at': None, 'district': '12', 'spo...",NaN,...,Puppies Assisting Wounded Servicemembers (PAWS...,"{'district': '6', 'type': 'person', 'title': '...",REFERRED,2016-03-16,"['Armed forces and national security', 'Health...",Armed forces and national security,"{'date': '2016-03-16T04:00:00Z', 'text': 'Pupp...",[{'title': 'Puppies Assisting Wounded Servicem...,2017-12-11T21:02:18Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...
2,"[{'acted_at': '2015-01-13', 'references': [], ...",[],hr337-114,hr,False,[],"[{'committee_id': 'HSPW', 'activity': ['referr...",114,[],NaN,...,Maritime Lien Reform Act,"{'district': None, 'type': 'person', 'title': ...",REFERRED,2015-01-13,"['Civil actions and liability', 'Licensing and...",Transportation and public works,"{'date': '2015-01-13T05:00:00Z', 'text': ""Mari...","[{'title': 'Maritime Lien Reform Act', 'type':...",2017-12-11T19:10:21Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...
3,"[{'acted_at': '2016-09-14', 'references': [], ...",[],hr6033-114,hr,False,[],"[{'committee_id': 'HSIF', 'activity': ['referr...",114,"[{'withdrawn_at': None, 'district': '13', 'spo...",NaN,...,Adding Middle East Respiratory Syndrome to the...,"{'district': '52', 'type': 'person', 'title': ...",REFERRED,2016-09-14,['Health'],Health,"{'date': '2016-09-14T04:00:00Z', 'text': 'Addi...",[{'title': 'Adding Middle East Respiratory Syn...,2017-12-11T21:27:52Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...
4,"[{'acted_at': '2015-05-21', 'references': [], ...",[],hr2493-114,hr,False,[],"[{'committee_id': 'HSAS', 'activity': ['referr...",114,"[{'withdrawn_at': None, 'district': '12', 'spo...",NaN,...,Wounded Warrior Service Dog Act of 2015,"{'district': '2', 'type': 'person', 'title': '...",REFERRED,2015-05-21,['Animal protection and human-animal relations...,Armed forces and national security,"{'date': '2015-05-21T04:00:00Z', 'text': 'Woun...",[{'title': 'Wounded Warrior Service Dog Act of...,2017-12-11T20:19:56Z,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...


### Create legislator database table.
The legislator table is flat and simply a copy of the csv file downloaded, so I will use `pandas` function `to_sql` for simplicity.

[Legislator Data Glossary](https://github.com/unitedstates/congress-legislators)

In [10]:
leg_hist = import_csv(os.path.join(data_path, 'legislators-historical.csv'))
leg_hist['current_legislator'] = False 
leg_hist.head()

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id,current_legislator
0,Bassett,Richard,NaN,NaN,NaN,NaN,1745-04-02,M,sen,DE,...,NaN,NaN,NaN,401222,NaN,NaN,NaN,507.0,Richard Bassett (politician),False
1,Bland,Theodorick,NaN,NaN,NaN,NaN,1742-03-21,M,rep,VA,...,NaN,NaN,NaN,401521,NaN,NaN,NaN,786.0,Theodorick Bland (congressman),False
2,Burke,Aedanus,NaN,NaN,NaN,NaN,1743-06-16,M,rep,SC,...,NaN,NaN,NaN,402032,NaN,NaN,NaN,1260.0,Aedanus Burke,False
3,Carroll,Daniel,NaN,NaN,NaN,NaN,1730-07-22,M,rep,MD,...,NaN,NaN,NaN,402334,NaN,NaN,NaN,1538.0,Daniel Carroll,False
4,Clymer,George,NaN,NaN,NaN,NaN,1739-03-16,M,rep,PA,...,NaN,NaN,NaN,402671,NaN,NaN,NaN,1859.0,George Clymer,False


In [11]:
leg_current = import_csv(os.path.join(data_path, 'legislators-current.csv'))
leg_current['current_legislator'] = True
leg_current.head()

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id,current_legislator
0,Brown,Sherrod,NaN,NaN,NaN,Sherrod Brown,1952-11-09,M,sen,OH,...,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown,True
1,Cantwell,Maria,NaN,NaN,NaN,Maria Cantwell,1958-10-13,F,sen,WA,...,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell,True
2,Cardin,Benjamin,L.,NaN,NaN,Benjamin L. Cardin,1943-10-05,M,sen,MD,...,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin,True
3,Carper,Thomas,Richard,NaN,NaN,Thomas R. Carper,1947-01-23,M,sen,DE,...,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper,True
4,Casey,Robert,P.,Jr.,Bob,"Robert P. Casey, Jr.",1960-04-13,M,sen,PA,...,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.,True


In [12]:
leg_all = pd.concat([leg_current, leg_hist])

In [13]:
len(leg_current) + len(leg_hist) == len(leg_all.drop_duplicates())

True

In [14]:
#leg_all.to_csv(os.path.join(data_path, 'legislators-all.csv'))
leg_all = import_csv(os.path.join(data_path, 'legislators-all.csv'))

In [15]:
leg_all.head()

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id,current_legislator
0,Brown,Sherrod,NaN,NaN,NaN,Sherrod Brown,1952-11-09,M,sen,OH,...,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown,True
1,Cantwell,Maria,NaN,NaN,NaN,Maria Cantwell,1958-10-13,F,sen,WA,...,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell,True
2,Cardin,Benjamin,L.,NaN,NaN,Benjamin L. Cardin,1943-10-05,M,sen,MD,...,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin,True
3,Carper,Thomas,Richard,NaN,NaN,Thomas R. Carper,1947-01-23,M,sen,DE,...,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper,True
4,Casey,Robert,P.,Jr.,Bob,"Robert P. Casey, Jr.",1960-04-13,M,sen,PA,...,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.,True


In [90]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
#leg_all.to_sql('legislators', con=engine, if_exists='replace', index_label='id')

In [16]:
bill_type = ['hr', 'hres', 'hjres', 'hconres', 's', 'sres', 'sjres', 'sconres']
bill_type_df = pd.DataFrame({'name':bill_type})

In [379]:
#bill_type_df.to_sql('lk_bill_type', con=engine, index_label='id', dtype={"name": sqlalchemy.types.String(8)})

# Data Cleaning.

### Bill versions.

In [17]:
bill_versions = pd.read_csv(os.path.join(data_path, 'bill_version.csv'))
bill_versions.columns = ['title', 'code', 'definition', 'chamber']

In [18]:
bill_versions.head()

,title,code,definition,chamber
0,Amendment (Senate),AS,An alternate name for this version is Senate A...,Senate
1,Additional Sponsors (House),ASH,An alternate name for this version is House Sp...,House
2,Agreed to (House),ATH,An alternate name for this version is Agreed t...,House
3,Agreed to (Senate),ATS,An alternate name for this version is Agreed t...,Senate
4,Committee Discharged (House),CDH,An alternate name for this version is House Co...,House Senate


In [19]:
## insert data into database from Python (proof of concept - this won't be useful for big data, of course)
bill_versions.to_sql('bill_versions', con=engine, if_exists='replace', index_label='id')

#### Unpack sponsor dictionary.

In [20]:
sponsor_dict = data_agg['sponsor'].apply(ast.literal_eval).values.tolist()
sponsor_df = pd.DataFrame(sponsor_dict, index=data_agg.index)[['bioguide_id']]

In [21]:
data_agg = data_agg.merge(sponsor_df, left_index=True, right_index=True)
del data_agg['sponsor']

#### Clean bill title data.

In [22]:
data_agg[['short_title', 'official_title', 'popular_title']].head()

,short_title,official_title,popular_title
0,Maker-Taker Conflict of Interest Reform Act of...,To require the Securities and Exchange Commiss...,NaN
1,Puppies Assisting Wounded Servicemembers (PAWS...,To direct the Secretary of Veterans Affairs to...,NaN
2,Maritime Lien Reform Act,To provide limitations on maritime liens on fi...,NaN
3,Adding Middle East Respiratory Syndrome to the...,To expand the tropical disease product priorit...,NaN
4,Wounded Warrior Service Dog Act of 2015,To establish a grant program to encourage the ...,NaN


The `titles` dict in the aggregated table data is redundant, so we delete it.

In [23]:
del data_agg['titles']

#### Separating actions.

In [24]:
data_actions = data_agg[['actions', 'bill_id']].copy()
del data_agg['actions']

#### Separating committees.

In [25]:
data_committees = data_agg[['committees', 'bill_id']].copy()
del data_agg['committees']

#### Separating amendments.

In [26]:
data_amendments = data_agg[['amendments', 'bill_id']].copy()
del data_agg['amendments']

#### Separating committee reports.

In [27]:
data_committee_reports = data_agg[['committee_reports', 'bill_id']].copy()
del data_agg['committee_reports']

#### Separating cosponsorship.

In [28]:
data_cosponsors = data_agg[['cosponsors', 'bill_id']].copy()
del data_agg['cosponsors']

#### Unpack and separate bill history dictionary.

In [29]:
history_dict = data_agg['history'].apply(ast.literal_eval).values.tolist()
history_df = pd.DataFrame(history_dict, index=data_agg.index)

In [30]:
history_df = data_agg[['bill_id']].merge(history_df, right_index=True, left_index=True)

In [31]:
del data_agg['history']

#### Separate related bills.

In [32]:
data_related_bills = data_agg[['related_bills', 'bill_id']].copy()
del data_agg['related_bills']

#### Separate subjects.

In [33]:
data_subjects = data_agg[['subjects', 'bill_id']].copy()
del data_agg['subjects']

#### Separate summaries.

In [34]:
data_summary = data_agg[['summary', 'bill_id', 'congress', 'number', 'bill_type']].copy()
del data_agg['summary']

#### Separate enacted_as.

In [35]:
data_enacted_as = data_agg[['enacted_as', 'bill_id']].copy()
del data_agg['enacted_as']

In [36]:
data_agg.columns

Index(['bill_id', 'bill_type', 'by_request', 'congress', 'introduced_at',
       'number', 'official_title', 'popular_title', 'short_title', 'status',
       'status_at', 'subjects_top_term', 'updated_at', 'url', 'bioguide_id'],
      dtype='object')

### Convert appropriate columns to `date` objects

In [37]:
data_agg['status_at'] = pd.to_datetime(data_agg['status_at']).dt.date

In [38]:
data_agg['introduced_at'] = pd.to_datetime(data_agg['introduced_at']).dt.date

In [39]:
data_agg['updated_at'] = pd.to_datetime(data_agg['updated_at'])

### Insert remaining data into `bills` table of database.

In [40]:
data_agg.head()

,bill_id,bill_type,by_request,congress,introduced_at,number,official_title,popular_title,short_title,status,status_at,subjects_top_term,updated_at,url,bioguide_id
0,hr1216-114,hr,False,114,2015-03-03,1216,To require the Securities and Exchange Commiss...,NaN,Maker-Taker Conflict of Interest Reform Act of...,REFERRED,2015-03-03,Finance and financial sector,2017-12-11 19:28:52,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...,L000562
1,hr4764-114,hr,False,114,2016-03-16,4764,To direct the Secretary of Veterans Affairs to...,NaN,Puppies Assisting Wounded Servicemembers (PAWS...,REFERRED,2016-03-16,Armed forces and national security,2017-12-11 21:02:18,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...,D000621
2,hr337-114,hr,False,114,2015-01-13,337,To provide limitations on maritime liens on fi...,NaN,Maritime Lien Reform Act,REFERRED,2015-01-13,Transportation and public works,2017-12-11 19:10:21,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...,Y000033
3,hr6033-114,hr,False,114,2016-09-14,6033,To expand the tropical disease product priorit...,NaN,Adding Middle East Respiratory Syndrome to the...,REFERRED,2016-09-14,Health,2017-12-11 21:27:52,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...,P000608
4,hr2493-114,hr,False,114,2015-05-21,2493,To establish a grant program to encourage the ...,NaN,Wounded Warrior Service Dog Act of 2015,REFERRED,2015-05-21,Armed forces and national security,2017-12-11 20:19:56,https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/...,M000312


In [41]:
list(data_agg.columns)

['bill_id',
 'bill_type',
 'by_request',
 'congress',
 'introduced_at',
 'number',
 'official_title',
 'popular_title',
 'short_title',
 'status',
 'status_at',
 'subjects_top_term',
 'updated_at',
 'url',
 'bioguide_id']

In [807]:
col_names = ['bill_id', 'by_request', 'congress', 'introduced_at',
             'number', 'official_title', 'popular_title', 'short_title',
             'status_at', 'subjects_top_term', 'updated_at', 'url']

relational_cols = ['status', 'bioguide_id', 'bill_type']

data_path = '/Users/melissaferrari/Projects/repo/insight/data_collection'
file_name = 'agg_propublica_114hr.csv'
data_agg = import_csv(os.path.join(data_path, file_name))


In [810]:
data_agg.columns

Index(['actions', 'amendments', 'bill_id', 'bill_type', 'by_request',
       'committee_reports', 'committees', 'congress', 'cosponsors',
       'enacted_as', 'history', 'introduced_at', 'number', 'official_title',
       'popular_title', 'related_bills', 'short_title', 'sponsor', 'status',
       'status_at', 'subjects', 'subjects_top_term', 'summary', 'titles',
       'updated_at', 'url'],
      dtype='object')

## Postgres with psycopg2.

In [357]:
db_conn = psycopg2.connect("host={} dbname={} user={}".format(hostname,
                                                              dbname,
                                                              username))

In [275]:
#db_conn.close()

In [358]:
db_cursor = db_conn.cursor()

#### List tables in database.

In [278]:
db_cursor.execute("""SELECT table_name 
                     FROM information_schema.tables 
                     WHERE table_schema = 'public'""")
for table in db_cursor.fetchall():
    print(table)

('lk_status',)
('related_bills',)
('congresspeople',)
('bill_actions',)
('committees',)
('bill_committee',)
('action_committee',)
('subjects',)
('cosponsorship',)
('summaries',)
('lk_summary_source',)
('bill_text',)
('bills',)
('legislators',)


#### Basic table queries.

In [487]:
db_conn.rollback()

In [489]:
db_cursor.execute('SELECT ls.name FROM lk_status ls')
all_status = db_cursor.fetchall()
all_status

[('ENACTED:SIGNED',),
 ('PASS_OVER:HOUSE',),
 ('REPORTED',),
 ('REFERRED',),
 ('PROV_KILL:CLOTUREFAILED',),
 ('PROV_KILL:SUSPENSIONFAILED',),
 ('PASS_BACK:SENATE',),
 ('PROV_KILL:VETO',),
 ('CONFERENCE:PASSED:HOUSE',),
 ('FAIL:ORIGINATING:HOUSE',),
 ('VETOED:OVERRIDE_FAIL_ORIGINATING:HOUSE',),
 ('ENACTED:TENDAYRULE',)]

In [293]:
db_cursor.execute('SELECT ls.full_name FROM legislators ls;')
all_status = db_cursor.fetchmany(10)
all_status

[('Sherrod Brown',),
 ('Maria Cantwell',),
 ('Benjamin L. Cardin',),
 ('Thomas R. Carper',),
 ('Robert P. Casey, Jr.',),
 ('Dianne Feinstein',),
 ('Amy Klobuchar',),
 ('Robert Menendez',),
 ('Bernard Sanders',),
 ('Debbie Stabenow',)]

#### Formatting SQL queries to load data into database.

In [103]:
#db_conn.rollback()

In [775]:
index = 2 #1226
row = data_agg.iloc[index]

In [776]:
col_names

['bill_id',
 'by_request',
 'congress',
 'introduced_at',
 'number',
 'official_title',
 'popular_title',
 'short_title',
 'status_at',
 'subjects_top_term',
 'updated_at',
 'url']

In [758]:
col_names = list(data_agg.columns)

In [766]:
relational_cols = ['status', 'bioguide_id', 'bill_type']

In [768]:
data_agg = data_agg[col_names + relational_cols];

In [760]:
[col_names.remove(f) for f in relational_cols]

[None, None, None]

In [769]:
col_names

['bill_id',
 'by_request',
 'congress',
 'introduced_at',
 'number',
 'official_title',
 'popular_title',
 'short_title',
 'status_at',
 'subjects_top_term',
 'updated_at',
 'url']

In [770]:
cols_string = str(col_names)[1:-1]
cols_string = cols_string.replace("'", '')
cols_string

'bill_id, by_request, congress, introduced_at, number, official_title, popular_title, short_title, status_at, subjects_top_term, updated_at, url'

In [656]:
vals_type = '%s, ' * len(col_names)
vals_type = vals_type[:-2]
vals_type

'%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s'

In [657]:
insert_list = list(row[col_names]) + list(row[related_cols])
insert_list

['hr337-114',
 False,
 114,
 datetime.date(2015, 1, 13),
 337,
 'To provide limitations on maritime liens on fishing permits, and for other purposes.',
 nan,
 'Maritime Lien Reform Act',
 datetime.date(2015, 1, 13),
 'Transportation and public works',
 Timestamp('2017-12-11 19:10:21'),
 'https://www.gpo.gov/fdsys/bulkdata/BILLSTATUS/114/hr/BILLSTATUS-114hr337.xml',
 'REFERRED',
 'Y000033',
 'hr']

In [658]:
for ix in range(len(insert_list)):
    if type(insert_list[ix]) == numpy.int64:
        insert_list[ix] = int(insert_list[ix])
    if type(insert_list[ix]) == numpy.bool_:
        insert_list[ix] = bool(insert_list[ix])

In [773]:
data_path

'/Users/melissaferrari/Projects/repo/insight/data_collection'

In [774]:
data_agg.columns

Index(['bill_id', 'by_request', 'congress', 'introduced_at', 'number',
       'official_title', 'popular_title', 'short_title', 'status_at',
       'subjects_top_term', 'updated_at', 'url', 'status', 'bioguide_id',
       'bill_type'],
      dtype='object')

In [771]:
related_cols_string = str(related_cols)[1:-1]
related_cols_string = related_cols_string.replace("'", '')

string_mapping = {"bioguide_id":"sponsor"}


In [663]:
for key in string_mapping.keys():
    related_cols_string = related_cols_string.replace(key, string_mapping[key])
query_keys = related_cols_string.split(', ')

'status, sponsor, bill_type'

In [751]:
index = []
column_name = []
for ix, val in enumerate(query_keys):
    index.append(ix)
    column_name.append(val)
table_relate = pd.DataFrame({'column':column_name}, index=index)
table_relate['relation_column'] = ['name', 'bioguide_id', 'name']
table_relate['relation_table'] = ['lk_status', 'legislators', 'lk_bill_type']

In [753]:
query_list = ''
for ix, row in table_relate.iterrows():
    query_list += '(SELECT ls.id FROM {} ls WHERE {}=%s),'.format(row['relation_table'], row['relation_column'])
query_list = query_list[:-1]    

In [754]:
sql = "INSERT INTO bills ({}, {}) VALUES ({}, {})".format(cols_string, 
                                                          related_cols_string, 
                                                          vals_type, 
                                                          query_list)
sql

'INSERT INTO bills (bill_id, by_request, congress, introduced_at, number, official_title, popular_title, short_title, status_at, subjects_top_term, updated_at, url, status, sponsor, bill_type) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, (SELECT ls.id FROM lk_status ls WHERE name=%s),(SELECT ls.id FROM legislators ls WHERE bioguide_id=%s),(SELECT ls.id FROM lk_bill_type ls WHERE name=%s))'

In [746]:
#db_conn.rollback()

In [749]:
db_cursor.execute(sql, insert_list)

In [750]:
db_conn.commit()

### Summary database.

In [944]:
data_summary['summary'][data_summary['summary'].isna()]

710     NaN
1443    NaN
2968    NaN
3927    NaN
5442    NaN
6491    NaN
Name: summary, dtype: object

In [945]:
data_summary.loc[data_summary['summary'].isna(), 'summary'] = "{'as': 'nan'}"

In [946]:
summary_dict = data_summary['summary'].apply(ast.literal_eval).values.tolist()

In [947]:
summary_df = pd.DataFrame(summary_dict, index=data_agg.index)

In [948]:
data_summary = data_summary.merge(summary_df, left_index=True, right_index=True)
del data_summary['summary']

In [949]:
data_summary['date'] = pd.to_datetime(data_summary['date']).dt.date

In [950]:
list(data_summary.columns)

['bill_id', 'congress', 'number', 'bill_type', 'as', 'date', 'text']

In [43]:
col_names = ['as', 'date', 'text']

relational_cols = ['bill_id', 'source']

data_path = '/Users/melissaferrari/Projects/repo/insight/data_collection'
file_name = 'agg_propublica_114hr.csv'
data_agg = import_csv(os.path.join(data_path, file_name))

data_summary = data_agg[['summary', 'bill_id']].copy()
data_summary.loc[data_summary['summary'].isna(),
                 'summary'] = "{'as': 'nan'}"

data_summary = data_summary.dropna()

In [44]:
to_dict = ast.literal_eval
summary_dict = data_summary['summary'].apply(to_dict).values.tolist()

summary_df = pd.DataFrame(summary_dict, index=data_agg.index)
data_summary = data_summary.merge(summary_df,
                                  left_index=True,
                                  right_index=True)
del data_summary['summary']
data_summary['source'] = 'CRS'

data_summary['date'] = pd.to_datetime(data_summary['date']).dt.date


In [45]:
data_summary[data_summary['bill_id'] == 'hr6061-114']['as']

710    nan
Name: as, dtype: object

In [46]:
data_summary.columns

Index(['bill_id', 'as', 'date', 'text', 'source'], dtype='object')

In [47]:
data_summary[data_summary['bill_id'] == 'hr2-114']

,bill_id,as,date,text,source
3433,hr2-114,Public Law,2015-04-17,(This measure has not been amended since it wa...,CRS


In [889]:
text = data_summary[data_summary['bill_id'] == 'hr2-114']['text'].values

In [890]:
type(text)

numpy.ndarray

In [895]:
text[0];

In [48]:
data_summary['as'].unique()

array(['Introduced in House', 'Passed House without amendment',
       'Passed House amended', 'Reported to House without amendment',
       'Public Law', 'Reported to Senate without amendment',
       'Reported to House with amendment(s)',
       'Reported to House amended, Part I',
       'Reported to House without amendment, Part I',
       'Reported to Senate with amendment(s)', 'nan',
       'Passed Senate amended', 'Conference report filed in House',
       'Reported to House amended, Part II',
       'Reported to House without amendment, Part II'], dtype=object)

In [ ]:
version_code = {'Introduced in House':'IH', 
                'Passed House without amendment',
                'Passed House amended', 
                'Reported to House without amendment',
                'Public Law', 
                'Reported to Senate without amendment',
                'Reported to House with amendment(s)',
                'Reported to House amended, Part I',
                'Reported to House without amendment, Part I',
                'Reported to Senate with amendment(s)', 
                'Passed Senate amended',
                'Conference report filed in House',
                'Reported to House amended, Part II',
                'Reported to House without amendment, Part II'}

In [922]:
bill_versions.sort_values('title', ascending=False);

In [953]:
len(data_summary[data_summary['as'] == 'Introduced in House'])

5542

In [959]:
import zipfile

In [975]:
for ix, row in data_summary[data_summary['as'] == 'Introduced in House'].iterrows():
    congress = row.congress
    bill_type = row.bill_type
    number = row.number
    bill_path = '/Users/melissaferrari/Projects/repo/congress/data/{}/bills/{}/{}{}/text-versions/ih/'.format(congress, bill_type, bill_type, number)

    try:
        os.path.isfile(os.path.join(bill_path, 'package.zip')) 
        zip_ref = zipfile.ZipFile(os.path.join(bill_path, 'package.zip'), 'r')
        zip_ref.extractall(bill_path)
        zip_ref.close()
    except:
        print(number)
        pass


5394


In [50]:
data_summary.columns

Index(['bill_id', 'as', 'date', 'text', 'source'], dtype='object')

In [51]:
data_summary.head()

,bill_id,as,date,text,source
0,hr1216-114,Introduced in House,2015-03-03,Maker-Taker Conflict of Interest Reform Act of...,CRS
1,hr4764-114,Introduced in House,2016-03-16,Puppies Assisting Wounded Servicemembers (PAWS...,CRS
2,hr337-114,Introduced in House,2015-01-13,Maritime Lien Reform Act\n\nThis bill revises ...,CRS
3,hr6033-114,Introduced in House,2016-09-14,Adding Middle East Respiratory Syndrome to the...,CRS
4,hr2493-114,Introduced in House,2015-05-21,Wounded Warrior Service Dog Act of 2015 Direct...,CRS


In [53]:
bill_ids = list(data_summary[data_summary['as'] == 'Introduced in House'].bill_id.values)

In [67]:
numbers = [int(f.split('hr')[1].split('-114')[0]) for f in bill_ids]

In [68]:
numbers

[1216,
 4764,
 337,
 6033,
 2493,
 3541,
 6201,
 3773,
 939,
 1024,
 105,
 4556,
 3317,
 1440,
 561,
 4132,
 1672,
 4300,
 753,
 3919,
 6457,
 3125,
 2231,
 5828,
 5014,
 595,
 1686,
 2003,
 4790,
 5642,
 2467,
 2655,
 3787,
 5470,
 3926,
 6468,
 4931,
 3328,
 4569,
 1827,
 1229,
 5689,
 2800,
 5219,
 798,
 5817,
 2652,
 3780,
 5477,
 4797,
 5645,
 1681,
 5221,
 2838,
 2004,
 2236,
 5013,
 592,
 754,
 4307,
 3122,
 3310,
 1447,
 4135,
 566,
 6206,
 3774,
 4551,
 102,
 5483,
 1211,
 330,
 4763,
 6034,
 3546,
 5810,
 2807,
 5448,
 3579,
 6239,
 2699,
 4936,
 559,
 3921,
 4338,
 6492,
 2032,
 796,
 5217,
 1485,
 5819,
 5025,
 5441,
 4593,
 2664,
 2456,
 5673,
 1015,
 908,
 1829,
 4567,
 134,
 6230,
 3742,
 2690,
 6002,
 3570,
 1227,
 306,
 6466,
 3928,
 1643,
 762,
 4331,
 4103,
 550,
 3326,
 2469,
 3789,
 5826,
 1688,
 5228,
 3319,
 4900,
 6459,
 3917,
 1218,
 339,
 937,
 4558,
 4938,
 1476,
 557,
 4104,
 3321,
 6461,
 3113,
 6005,
 3577,
 1220,
 5680,
 4752,
 301,
 1012,
 133,
 4560,
 6

### Populating bill text.

In [80]:
import re

In [70]:
congress = '114'
bill_type = 'hr'
default_bill_number = 4764
#2, 34, 876, 1321, 1624, 2212, 2820, 3831
#number = bill_numbers[5]

In [71]:
number = default_bill_number

In [61]:
bill_path_root = '/Users/melissaferrari/Projects/repo/congress/data/'

In [62]:
xml_paths = [x[0] for x in os.walk(bill_path_root) if 'xml' in x[0]]

In [78]:
xml_path = xml_paths[0]
xml_path

'/Users/melissaferrari/Projects/repo/congress/data/114/bills/hr/hr1216/text-versions/ih/BILLS-114hr1216ih/xml'

In [81]:
regex_pattern_00 = re.compile('\d\d\d([a-z]+)\d+')
match_00 = regex_pattern_00.search(xml_path).group()

regex_pattern_01 = re.compile('[a-z]+')
bill_type = regex_pattern_01.search(match_00).group()
congress, number = match_00.split(bill_type)
bill_id = '{}{}-{}'.format(bill_type, number, congress)

In [82]:
bill_id

'hr1216-114'

In [72]:
bill_path = '{}{}/bills/{}/{}{}/text-versions/ih/'.format(bill_path_root, 
                                                          congress, 
                                                          bill_type, 
                                                          bill_type, 
                                                          number)
bill_path += 'BILLS-{}{}{}ih/xml'.format(congress, bill_type, number)

xml_file = os.listdir(bill_path)[0]

In [74]:
bill_path

'/Users/melissaferrari/Projects/repo/congress/data/114/bills/hr/hr4764/text-versions/ih/BILLS-114hr4764ih/xml'

In [77]:
xml_file = os.listdir(bill_path)[0]
xml_file

'BILLS-114hr4764ih.xml'